In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Импорт библиотек

In [ ]:
!pip install keras_preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers, models
from keras_preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model

from pathlib import Path
import os.path
import random

import matplotlib.cm as cm
import cv2
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
import itertools

import warnings
warnings.filterwarnings("ignore")

# Импорт датасета

In [ ]:
dataset_dir = '/kaggle/input/100-bird-species/'
train_dir = '/kaggle/input/100-bird-species/train'
val_dir = '/kaggle/input/100-bird-species/valid'
test_dir = '/kaggle/input/100-bird-species/test'

In [ ]:
train_imgs_arr = [os.path.join(train_dir, filename) for filename in os.listdir(train_dir)]
val_imgs_arr = [os.path.join(val_dir, filename) for filename in os.listdir(val_dir)]
test_imgs_arr = [os.path.join(test_dir, filename) for filename in os.listdir(test_dir)]

При предобработке данных оказалось, что в датасете присутствует опечатка. Исправим ее, чтобы метка в датафрейме соответствовала названию папок датасета.

In [ ]:
df_all = pd.read_csv('/kaggle/input/100-bird-species/birds.csv')

df_all.loc[df_all['labels'] == 'PARAKETT  AKULET', 'labels'] = 'PARAKETT  AUKLET'
for i in range(len(df_all['filepaths'])):
    row = df_all['filepaths'][i]
    if 'PARAKETT  AKULET' in row:
        row_list = row.split('/')
        if row_list[0] == 'valid':
            df_all['filepaths'][i] = os.path.join(row_list[0], 'PARAKETT AUKLET', row_list[-1])
        else:
            df_all['filepaths'][i] = os.path.join(row_list[0], 'PARAKETT  AUKLET', row_list[-1])


df_train = df_all.loc[df_all['data set'] == 'train']
df_train = df_train.reset_index(drop=True)

df_val = df_all.loc[df_all['data set'] == 'valid']
df_val = df_val.reset_index(drop=True)

df_test = df_all.loc[df_all['data set'] == 'test']
df_test = df_test.reset_index(drop=True)

print('Размер train: ', df_train.shape)
print('Размер val: ', df_val.shape)
print('Размер test: ', df_test.shape)

In [ ]:
df_parakett = df_all.loc[df_all['labels'] == 'PARAKETT  AKULET']
df_parakett.shape

In [ ]:
df_train.head()

In [ ]:
df_train.isnull().sum()

Приведем метки классов к целочисленному типу

In [ ]:
df_train['class id'] = df_train['class id'].apply(int)
df_val['class id'] = df_val['class id'].apply(int)
df_test['class id'] = df_test['class id'].apply(int)
df_train.head()

In [ ]:
for col in df_train.columns.tolist():
    print(f'Уникальных значений в столбце {col}: {len(df_train[col].unique())}')

Заметим, что уникальных научных названий меньше, чем неофициальных. Так как пропусков нет, на некоторые научные названия приходится несколько неофициальных. 

Составим словарь, где ключами будут метки класса, а значениями -- названия видов птиц, а также словарь, где ключами будут названия видов птиц, а значениями -- научные названия видов птиц

In [ ]:
name_class = {}
def name_matching(row):
    name_class[row['class id']] = row['labels']
df_val.apply(name_matching, axis=1)
len(name_class)

In [ ]:
class_name_dict = {}
def class_matching(row):
    class_name_dict[row['labels']] = row['class id']
df_val.apply(class_matching, axis=1)
len(class_name_dict)

In [ ]:
name_sci_name = {}
def sci_name_matching(row):
    name_sci_name[row['labels']] = row['scientific name']
df_val.apply(sci_name_matching, axis=1)
len(name_sci_name)

# Просмотр изображений

In [ ]:
random_index = np.random.randint(0, len(df_train), 16)
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(dataset_dir + df_train['filepaths'][random_index[i]]))
    ax.set_title(df_train['labels'][random_index[i]])
plt.tight_layout()
plt.show()

In [ ]:
# Проредим список для наглядности
all_label_counts = df_train['labels'].value_counts()
bar_labels = all_label_counts[::100]

plt.figure(figsize=(20, 6))
sns.barplot(x=bar_labels.index, y=bar_labels.values, alpha=0.8)
plt.title('Распределение птиц по классам', fontsize=16)
plt.xlabel('Вид', fontsize=14)
plt.ylabel('Количество', fontsize=14)
plt.xticks(rotation=45)
plt.show()

In [ ]:
print(f'Самый многочисленный класс представлен в выборке в {round(all_label_counts.max() / all_label_counts.min(), 2)} раза больше, чем самый малочисленный')

Заметим, что в датасете наблюдается серьезная диспропорция классов. На это необходимо будет обратить внимание при выборе модели.

Или же можно решить эту проблему аугментацией изображений для малопредставленных в обучающей выборке классов. Будем исходить из того, что в реальных задачах вероятность получить на вход модели фотографии разных видов птиц будет одинакова для всех классов. 

Построим пайплайн для аугментации:

In [ ]:
aug_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

Напишем функцию для аугментации данных только для тех классов, количество изображений в которых меньше значения целевого параметра аугментации

In [ ]:
def augment_images(class_name, target_count, data_dir, save_dir, datagen):
    if class_name == 'PARAKETT AUKLET':
        if data_dir == '/kaggle/input/100-bird-species/train':
            class_dir = '/kaggle/input/100-bird-species/train/PARAKETT  AUKLET'
        if data_dir == '/kaggle/input/100-bird-species/test':
            class_dir = '/kaggle/input/100-bird-species/test/PARAKETT  AUKLET'
        else:
            class_dir == '/kaggle/input/100-bird-species/valid/PARAKETT AUKLET'
    else:
        class_dir = os.path.join(data_dir, class_name)
    images = os.listdir(class_dir)
    current_count = len(images)
    images_needed = target_count - current_count

    if images_needed <= 0:
        return

    print(f"Аугментируем {images_needed} изображений для класса {class_name}")
    
    class_save_dir = os.path.join(save_dir, class_name)
    os.makedirs(class_save_dir, exist_ok=True)

    for i in range(images_needed):
        image_path = os.path.join(class_dir, images[i % current_count])
        img = load_img(image_path)
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Generate a batch of one image
        augmented_iter = datagen.flow(x, batch_size=1)

        # Save the generated image
        save_path = os.path.join(class_save_dir, f"aug_{i}.jpg")
        aug_img = next(augmented_iter)[0].astype(np.uint8)
        aug_img = np.squeeze(aug_img)
        tf.keras.preprocessing.image.save_img(save_path, aug_img)

In [ ]:
data_dir = train_dir
class_name = 'ABBOTTS BABBLER'
class_dir = os.path.join(data_dir, class_name)
images = os.listdir(class_dir)
print(class_dir)
len(images)

In [ ]:
target_count = all_label_counts.max()
data_dir = train_dir
save_dir = '/kaggle/working/train'
datagen = aug_datagen
if not os.path.exists(save_dir):
    for class_name in df_train['labels'].unique():
            augment_images(class_name, target_count, data_dir, save_dir, datagen)

In [ ]:
augmented_dir = '/kaggle/working/train'
augmented_data = []

# Пройдемся по всем папкам и файлам в директории аугментированных изображений
for class_name in os.listdir(augmented_dir):
    class_dir = os.path.join(augmented_dir, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            augmented_data.append({'filepaths': img_path, 'labels': class_name})

df_aug = pd.DataFrame(augmented_data)
df_aug.head()

In [ ]:
df_aug.info()

Оставим от датасетов только метки класса и путь к изображению птицы

In [ ]:
df_train = df_train[['filepaths', 'labels']]
df_train['filepaths'] = df_train['filepaths'].apply(lambda x: os.path.join(dataset_dir, x))

df_val = df_val[['filepaths', 'labels']]
df_val['filepaths'] = df_val['filepaths'].apply(lambda x: os.path.join(dataset_dir, x))

df_test = df_test[['filepaths', 'labels']]
df_test['filepaths'] = df_test['filepaths'].apply(lambda x: os.path.join(dataset_dir, x))

df_train.head()

In [ ]:
df_train_orig = df_train.copy()

In [ ]:
df_train = pd.concat([df_train_orig, df_aug], ignore_index=True)
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_val.head()

При помощи аугментации изображений были сбалансированы классы. Теперь каждый класс представляет 263 изображений.

Создадим загрузчики данных для каждого датасета

In [ ]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

val_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

In [ ]:
for col in df_train.columns.tolist():
    print(f'Уникальных значений в столбце {col}: {len(df_train[col].unique())}')
    
for col in df_val.columns.tolist():
    print(f'Уникальных значений в столбце {col}: {len(df_val[col].unique())}')
    
for col in df_test.columns.tolist():
    print(f'Уникальных значений в столбце {col}: {len(df_test[col].unique())}')

In [ ]:
def check_file_existence(df):
    missing_files = df[~df['filepaths'].apply(lambda x: os.path.exists(x))]
    if not missing_files.empty:
        print(f"Missing files: {len(missing_files)}")
        print(missing_files)
    else:
        print("No missing files.")

print("Checking training data...")
check_file_existence(df_train)

print("Checking validation data...")
check_file_existence(df_val)

print("Checking test data...")
check_file_existence(df_test)


In [ ]:
batch_size = 32
target_size = (224, 224)

train_images = train_generator.flow_from_dataframe(
    dataframe=df_train,
    x_col='filepaths',
    y_col='labels',
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

val_images = val_generator.flow_from_dataframe(
    dataframe=df_val,
    x_col='filepaths',
    y_col='labels',
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

test_images = test_generator.flow_from_dataframe(
    dataframe=df_test,
    x_col='filepaths',
    y_col='labels',
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
)